# 🤗 Episodio 5 - RoBERTa V2 - 3 epochs


In [4]:
import utils
import importlib
importlib.reload(utils)

from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from utils import get_submission, submit, get_tokenizer_and_model,\
                  compute_metrics, load_dfs, tokenize, get_train_args


import transformers
from transformers import Trainer
transformers.logging.set_verbosity_warning()

def read_scores():
    s = !kaggle competitions submissions nlp-getting-started 
    df = pd.DataFrame([(l.split()[0], l.split()[-2]) for l in s[2:]], columns=["Archivo", "Score"]).set_index("Archivo")
    display(df)
    return df

utils.nb_set_width()

In [5]:
model_id = "episodio_5_v2"
pred_file_name = f"{model_id}.csv"
submit_message = "Episodio 5 RoBERTa Base 3 epochs"

model_name = "roberta-base"
tokenizer, model = get_tokenizer_and_model(model_name)

df_base, df_test = load_dfs()
df_train, df_val = train_test_split(df_base, test_size=0.066)
ds_train, ds_val, ds_test = tokenize(tokenizer, df_train, df_val, df_test)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifi

In [6]:
trainer = Trainer(model=model, args=get_train_args(model_id, epochs=3, save_steps=500, evaluation_strategy="steps", 
                                                   eval_steps=200), 
                  train_dataset=ds_train, 
                  eval_dataset=ds_val,
                  compute_metrics=compute_metrics)

trainer.train()

Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
200,No log,0.486917,0.771372,0.650456,0.906780,0.507109
400,No log,0.557595,0.761431,0.638554,0.876033,0.502370
600,0.601300,0.630170,0.636183,0.687179,0.537433,0.952607
800,0.601300,0.446269,0.838966,0.798005,0.842105,0.758294
1000,0.597100,0.489709,0.836978,0.797030,0.834197,0.763033
1200,0.597100,0.513146,0.795229,0.778495,0.712598,0.857820
1400,0.597100,0.483765,0.829026,0.792271,0.807882,0.777251
1600,0.522800,0.439466,0.821074,0.784689,0.792271,0.777251
1800,0.522800,0.444494,0.811133,0.791209,0.737705,0.853081
2000,0.467300,0.409108,0.852883,0.819512,0.844221,0.796209


TrainOutput(global_step=2667, training_loss=0.5155977607443845, metrics={'train_runtime': 2351.3276, 'train_samples_per_second': 9.071, 'train_steps_per_second': 1.134, 'total_flos': 8167600546099200.0, 'train_loss': 0.5155977607443845, 'epoch': 3.0})

In [7]:
df_res = get_submission(trainer, ds_test, pred_file_name)

In [8]:
submit(pred_file_name, submit_message)

stdout=
Successfully submitted to Natural Language Processing with Disaster Tweets
stderr=
100%|██████████| 22.2k/22.2k [00:00<00:00, 113kB/s]



In [11]:
# Leaderbord - 625 - 0.82562
read_scores();

,Score
Archivo,
episodio_5_v2.csv,0.82807
episodio_5_part1-4-epochs-cp2500.csv,0.82592
episodio_5_part1-4-epochs.csv,0.81918
episodio_5_part1.csv,0.82562
episodio_5.csv,0.57033
episodio_4_pred_4.csv,0.81979
episodio_4_pred_2_checkpoint_2000.csv,0.80968
episodio_4_pred_2.csv,0.81734
episodio_4_pred_1.csv,0.81274


In [12]:
625 - 576

49

In [13]:
576 - 141

435

In [14]:
576 / 2600

0.22153846153846155

In [15]:
tokenizer

PreTrainedTokenizerFast(name_or_path='roberta-base', vocab_size=50265, model_max_len=512, is_fast=True, padding_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=False)})